In [1]:
import pandas as pd
from datetime import datetime

In [2]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [3]:
cols=['Name','Market Cap (US$ Billion)']

In [4]:
def extract_market_cap():
    df = pd.DataFrame(cols)
    df = extract_from_json("bank_market_cap.json")
    return df

In [5]:
def extract_exchange_rates():
    exchange_rate = pd.read_csv("exchange_rates_1.csv")
    exchange_rate = exchange_rate.rename(columns={"Unnamed: 0":"Currency"})
    return exchange_rate

In [13]:
def transform(market_cap, currency):
    all_rates = extract_exchange_rates()
    rate = all_rates.loc[lambda all_rates: all_rates['Currency'] == currency]['rates']
    market_cap["Market Cap (US$ Billion)"] = round(market_cap["Market Cap (US$ Billion)"].multiply(rate.iloc[0]), 3)
    market_cap = market_cap.rename(columns={"Market Cap (US$ Billion)":"Market Cap ({} Billion)".format(currency)})
    return market_cap

In [7]:
def load(bank_market_cap, currency):
    bank_market_cap.to_csv("bank_market_cap_{}.csv".format(currency), index=False)
    

In [8]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [9]:
#get user input
def get_currency():
    user_input = input("Enter a currency that you would like to see (Default = USD):")
    all_rates = extract_exchange_rates()
    currency = "USD"
    for rate in all_rates['Currency']:
        if (user_input == rate):
            currency = user_input
    return currency

In [10]:
# Logging started
log("ETL job started")

In [11]:
log("Extract phase started")
# Extracting data from market cap json
extracted_data = extract_market_cap()
log("Extract phase ended")
# Print the first 5 rows here
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [14]:
log("Transform phase started")
# Call the function here
currency = get_currency()
transformed_data = transform(extracted_data, currency)
log("Transform phase ended")
# Print the first 5 rows here
transformed_data.head()

,Name,Market Cap (AUD Billion)
0,JPMorgan Chase,646.732
1,Industrial and Commercial Bank of China,571.096
2,Bank of America,538.204
3,Wells Fargo,509.554
4,China Construction Bank,425.822


In [13]:
log("Load phase started")
# loading into csv
load(transformed_data, currency)
log("Load phase ended")

In [14]:
#Logging ended
log("ETL job ended")